In [1]:
%matplotlib inline
import numpy as np
import cv2
import matplotlib.pyplot as plt

### Teorização:
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Para amenizar oscilações na camera, é necessário capturar o fluxo óptico entre dois frames, porém, isso não basta, uma vez que ele só retorna o quanto cada um dos pontos escolhidos variou em relação à sua posição anterior. Para estabilizar a camera de fato, é necessário que o movimento dos pontos seja compensado com uma translação na direção oposta. Agora, há a questão dos pontos escolhidos pelo goodFeaturesToTtrack, pois se a seleção for feita com a tela inteira, a estabilização tentaria centralizar a imagem inteira, minimizando os efeitos, sendo que a imagem em sí sempre estaria no centro. Por isso, o código da aula 13 não é eficaz na estabilização.\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Uma maneira de contornar esse problema é limitar a janela de pontos do goodFeaturesToTrack para o centro da camera e diminuir o tamanho da imagem mostrada, assim, a imagem mostrada pode transladar pela imagem total capturada pela camera e sempre será compensada para o centro.\

Com os detalhes iniciais acertados, é iniciado o processo de determinação do fluxo ótico

In [ ]:
cap = cv2.VideoCapture(0)


# Parametros para a detecção de cantos de Shi-Tomasi
feature_params = {
    'maxCorners': 100,
    'qualityLevel': 0.3,
    'minDistance': 7,
    'blockSize': 7,
    'qualityLevel': 0.01
}

# Parametros para o fluxo ótico de Lucas-Kanade
lk_params = {
    'winSize': (15, 15),
    'maxLevel': 2,
    'criteria': (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
}

# Cores aleatórias para os pontos
color = np.random.randint(0, 255, (100, 3))


_, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
rows, cols = old_gray.shape
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params) # Cálculo das primeiras goodFeaturesToTrack

size = 70 # tamanho do quadrado onde os pontos serão pegos (2xsize por 2xsize), valores altos se comportam de maneira inesperada

# limitação dos pontos para a janela central, removendo qualquer um que estiver fora, no primeiro frame
p0 = p0[
    (p0[:,:,0] >= cols/2-size) &
    (p0[:,:,0] <= cols/2+size) &
    (p0[:,:,1] >= rows/2-size) &
    (p0[:,:,1] <= rows/2+size)
].reshape(-1, 1, 2) # -> remontar o shape para o goodFeatures

# Máscara para o desenho
mask = np.zeros_like(old_frame)

### Implementação
\
Equacao do fluxo otico:  
$$
I(x, y, t) = I(x + \Delta x, y + \Delta y, t + \Delta t)
$$
\
Após a decomposição em série de Taylor, a equação fica:
$$
I_x v_x + I_y v_y + I_t = 0
$$
Nessa solução do problema de estabilização de camera, foi empregado o método de Lucas-Kanade para calcular o fluxo ótico. Esse é um método diferencial para estimá-lo, a partir da suposição de que o fluxo (translação) é relativamente constante ao redor de um certo pixel. Asism, ele aplica as equações do fluxo para cada ponto e se utilizad od método dos mínimos quadrados para resolver o sistema de equações.

Outro problema que foi abordado foi o dasbordas pretas que surgem caso a imagem seja transladada demais em algum dos eixos. Para contornar isso, além de só dar display de uma janela menor que a imagem completa, também foi implementado uma maneira de impedir que a janela mostrada ultrapasse as bordas da imagem completa. Para tal, bastou calcular a distancia máxima entre o fim da janela e a borda da imagem e utilizar esse valor como limitante da translação.

In [4]:
dx = 0 # coeficiente da translação na coordenada X
dy = 0 # coeficiente da translação na coordenada Y

# Para impedir a aparição de bordas pretas na imagem tratada, são definidas as variações máximas em X e Y, 
max_x = cols//2-size*2
max_y = rows//2-size*2-60 # 60 pixels em y, em cima e em baixo da imagem, por conta da camera

while True:
    _, frame = cap.read()
    if frame is None:
        break
        
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Cálculo do fluxo ótico
    p1, st, err = cv2.calcOpticalFlowPyrLK(
        old_gray, frame_gray, p0, None, **lk_params)

    
    # Cláusula catch para impedir que o kernel quebre se todos os pontos forem perdidos
    try:
        # Seleção dos pontos bons
        good_new = p1[st == 1]
        good_old = p0[st == 1]

        # Calcula a média da variação de todos os pontos do goodFeatures
        dx += (p0[:,:,0]-p1[:,:,0]).mean()
        dy += (p0[:,:,1]-p1[:,:,1]).mean()
        
    except:
        print("Todos os pontos foram perdidos, rodar denovo")
    
    # limitação dos deslocamentos máximos que impedem a aparição de bordas pretas
    if dx > max_x:
        dx = max_x
    if dy > max_y:
        dy = max_y
    if dx < -max_x:
        dx = -max_x
    if dy < -max_y:
        dy = -max_y
    
    M = np.array([
        [1.0, 0.0, dx],
        [0.0, 1.0, dy],
    ]) # matriz para o warpAffine
    
    frame_tranls = cv2.warpAffine(frame, M, None) # Translada a imagem, de acordo com os coeficientes dx e dy

    # Desenhos para melhor visualização dos pontos seguidos
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        #cv2.line(mask, (int(a+dx), int(b+dy)), (int(c+dx), int(d+dy)), color[i].tolist(), 2)
        cv2.circle(frame_tranls, (int(a+dx), int(b+dy)), 5, color[i].tolist(), -1)
        cv2.rectangle(frame_tranls, (cols//2 - size, rows//2 - size), (cols//2 + size, rows//2 + size), (0, 255, 0)) # // -> indices tem que ser inteiros
        #cv2.rectangle(frame_tranls, (cols//2 - size*2, rows//2 - size*2), (cols//2 + size*2, rows//2 + size*2), (255, 0, 0))
    img = cv2.add(frame_tranls, mask)

    # Display de parte cortada da imagem
    cv2.imshow('frame', img[rows//2-size*2:rows//2+size*2, cols//2-size*2:cols//2+size*2])

    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Atualização dos frames anteriores
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cap.release()
cv2.destroyAllWindows()